In [82]:
from pyscenic.rnkdb import FeatherRankingDatabase, RankingDatabase
from typing import Type
import os
import numpy as np
import pandas as pd
from feather.api import write_dataframe, FeatherReader
from tqdm import tqdm

In [75]:
DB_FOLDER = "/Users/bramvandesande/Projects/lcb/databases"

In [76]:
db = FeatherRankingDatabase(fname=os.path.join(DB_FOLDER, "hg19-regions-220330-9species.extracted.feather"),
                            name="regions", nomenclature="HGNC")

In [77]:
len(db.genes)

220329

In [85]:
IDENTIFIERS_FNAME_EXTENSION = "identifiers.txt"
INVERTED_DB_DTYPE = np.uint32
INDEX_NAME = "features"

In [79]:
def _derive_identifiers_fname(fname):
    return '{}.{}'.format(os.path.splitext(fname)[0], IDENTIFIERS_FNAME_EXTENSION)

In [86]:
def invert(db: Type[RankingDatabase], fname: str, top_n_identifiers: int = 50000) -> None:
    """
    Create an inverted whole genome rankings database keeping only the top n genes/regions for a feature.

    Inverted design: not storing the rankings for all regions in the dataframe but instead store the identifier of the
    top n genes/regions in the dataframe introduces an enormous reduction in disk and memory size.

    :param db: The rankings database.
    :param fname: the filename of the inverted database to be created.
    :param top_n_identifiers: The number of genes to keep in the inverted database.
    """

    df_original = db.load_full()
    n_features = len(df_original)

    index_fname = _derive_identifiers_fname(fname)
    assert not os.path.exists(index_fname), "Database index {0:s} already exists.".format(index_fname)
    identifiers = df_original.columns.values
    with open(index_fname, 'w') as f:
        f.write('\n'.join(identifiers))
    identifier2idx = {identifier: idx for idx, identifier in enumerate(identifiers)}

    inverted_data = np.empty(shape=(n_features, top_n_identifiers), dtype=INVERTED_DB_DTYPE)
    df_original.columns = [identifier2idx[identifier] for identifier in df_original.columns]
    for idx, (_, row) in tqdm(enumerate(df_original.iterrows())):
        inverted_data[idx, :] = np.array(row.sort_values(ascending=True).head(top_n_identifiers).index, dtype=INVERTED_DB_DTYPE)
    df = pd.DataFrame(data=inverted_data, index=df_original.index, columns=list(range(top_n_identifiers)))

    df.index.name = INDEX_NAME
    df.reset_index(inplace=True) # Index is not stored in feather format. https://github.com/wesm/feather/issues/200
    write_dataframe(df, fname)

In [88]:
invert(db, os.path.join(DB_FOLDER, "hg19-regions-220330-9species.inverted.feather"))

9713it [09:27, 17.13it/s]


In [34]:
nr_chars = max(map(len,df_.columns))
nr_chars

14

In [39]:
import numpy as np
import pandas as pd
df_ = df.head()
nr_chars = max(map(len,df_.columns))
TOP_N = 50000
n_features = len(df_)
inverted_data = np.empty(shape=(n_features, TOP_N), dtype='U{}'.format(nr_chars))
for idx, (_, row) in enumerate(df_.iterrows()):
    inverted_data[idx, :] = row.sort_values(ascending=True).head(TOP_N).index
inverted_df = pd.DataFrame(data=inverted_data, index=df_.index, columns=list(range(TOP_N)))

In [40]:
inverted_df

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
features,,,,,,,,,,,,,,,,,,,,,
elemento-AAAATGGCG,chrX-reg32698,chr3-reg11533,chr2-reg79638,chr3-reg110064,chr1-reg108107,chr7-reg58012,chr16-reg9028,chr11-reg40102,chr11-reg8174,chr11-reg9500,...,chr8-reg23479,chr16-reg45922,chr19-reg8404,chr12-reg77896,chr1-reg18112,chr1-reg28342,chr12-reg7113,chr5-reg8302,chr16-reg44441,chr17-reg70760
elemento-AAATCAAT,chr10-reg33634,chr9-reg32450,chr7-reg19391,chr2-reg80602,chr6-reg57419,chr13-reg43061,chr17-reg42372,chr14-reg3151,chr18-reg35521,chr14-reg50228,...,chr12-reg86983,chr7-reg70627,chr11-reg66224,chr22-reg4498,chr11-reg4974,chr15-reg9512,chr1-reg131822,chr7-reg62196,chr2-reg42740,chr17-reg33927
elemento-AAATGCAAA,chr17-reg60295,chr1-reg165040,chr10-reg66722,chr10-reg34533,chr7-reg2132,chr11-reg81814,chr20-reg16942,chr9-reg57882,chr11-reg91095,chr5-reg91502,...,chr17-reg635,chr1-reg42975,chr8-reg27484,chr11-reg52580,chr1-reg102464,chr11-reg73235,chr15-reg26297,chr5-reg112328,chr1-reg44410,chr21-reg22019
elemento-AAATTGCA,chr5-reg85184,chr17-reg33458,chr2-reg118522,chr10-reg94934,chr1-reg78218,chr2-reg116500,chr10-reg34533,chrX-reg32938,chrX-reg49548,chr1-reg89950,...,chr8-reg62041,chrX-reg47193,chr5-reg20272,chrX-reg5653,chr18-reg2385,chr19-reg22852,chr1-reg56518,chr9-reg66776,chr4-reg61821,chr14-reg15767
elemento-AACAGCTG,chr2-reg144559,chr2-reg114576,chr2-reg122253,chr11-reg61192,chr3-reg100749,chr9-reg28130,chr14-reg31649,chr15-reg7899,chr4-reg80579,chr10-reg9206,...,chr14-reg18607,chr20-reg48928,chr11-reg16370,chr14-reg59120,chr22-reg25707,chr20-reg5231,chr1-reg109841,chrX-reg11221,chr3-reg75807,chr2-reg103367


In [41]:
inverted_df.memory_usage(deep=True).sum()

17612575